In [3]:
# ==========================================
# 1. SETUP & SECURITY
# ==========================================
from crewai import Agent, Task, Crew
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os

# This loads your API keys from a hidden .env file so they aren't exposed in the code.
load_dotenv()

# We define the brain (LLM) once, so we can share it with all agents.
llm = ChatOpenAI(model="gpt-4o-mini")

# ==========================================
# 2. DEFINE THE TEAM (The Agents)
# ==========================================
# We represent a team of three distinct experts.

# Agent 1: The Researcher. Their only job is to go out and get the raw data.
fetcher = Agent(
    role="FetcherAgent",
    goal="Search and extract top papers relevant to a query topic",
    backstory="Expert at discovering high-quality research sources.",
    llm=llm,
)

# Agent 2: The Writer. They take raw data and make it readable.
summarizer = Agent(
    role="SummarizerAgent",
    goal="Read abstracts and extract the key findings and methods.",
    backstory="Academic writer skilled at condensing research.",
    llm=llm,
)

# Agent 3: The Editor. They grade the work to ensure quality control.
critic = Agent(
    role="CriticAgent",
    goal="Evaluate summaries for novelty, clarity, and potential bias.",
    backstory="Peer reviewer who checks academic accuracy.",
    llm=llm,
)

# ==========================================
# 3. DEFINE THE WORKFLOW (The Tasks)
# ==========================================
# In CrewAI, tasks are usually "Sequential". 
# The output of Task 1 automatically becomes the input context for Task 2.

# Step 1: Get the data.
task_fetch = Task(
    description="Retrieve 5 papers on 'large language model alignment' from trusted sources.",
    # We ask for JSON format so the next agent can easily read the data structure.
    expected_output="A JSON list with title, authors, and abstract text.",
    agent=fetcher,      
)

# Step 2: Process the data.
# The 'Summarizer' will automatically see the 'JSON list' created by the 'Fetcher'.
task_summarize = Task(
    description="Summarize the abstracts from the FetcherAgent output into concise key points.",
    expected_output="Summaries with key contributions and comparisons.",
    agent=summarizer,      
)

# Step 3: Review the data.
# The 'Critic' will see the summaries created by the 'Summarizer' and grade them.
task_critic = Task(
    description="Review the SummarizerAgent output and rate each summary on novelty and clarity (1–5).",
    expected_output="A table of paper titles, output of SummarizerAgent output, ratings, and comments.",
    agent=critic,   
)

# ==========================================
# 4. MANAGE THE PROCESS (The Crew)
# ==========================================
crew = Crew(
    # The order of agents in this list doesn't strictly matter, 
    # but the order of TASKS below is crucial.
    agents=[fetcher, summarizer, critic],
    
    # The order here defines the "Assembly Line".
    # fetch -> summarize -> critic
    tasks=[task_fetch, task_summarize, task_critic],
    
    # 'verbose=True' lets you see the agents "thinking" in the console as they work.
    verbose=True
)

# ==========================================
# 5. EXECUTION
# ==========================================
print("Starting the academic review process...")
result = crew.kickoff()

print("######################")
print("FINAL REPORT:")
print(result)

Starting the academic review process...


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: f628b98a-ac4b-4048-8603-c2251398017e                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: FetcherAgent                                                                                            │
│                                                                                                                 │
│  Task: Retrieve 5 papers on 'large language model alignment' from trusted sources.                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: FetcherAgent                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```json                                                                                                        │
│  [                                                                                                              │
│      {                                                                                                          │
│          "title": "Language Models as Knowledge Bases?",                                                        │
│          "authors": ["Gururangan, S.", "Marasović, A.", "Swayamdipta, S.", "Lo, K.", "Smith, N. A.", "Manning,  │
│  C. D."],                                                                                                       │
│          "abstract": "The recent advancement of large pre-trained language models (PLMs) has opened up          │
│  exciting new possibilities for natural language processing (NLP). One interesting aspect of these models is    │
│  their capability to store and retrieve factual knowledge from their training data. In this paper, we           │
│  investigate the viability of leveraging PLMs as knowledge bases (KBs) and empirically analyze their            │
│  performance on various knowledge-based tasks. By comparing PLMs with traditional KBs, we demonstrate not only  │
│  the strengths but also the limitations of using these models in this capacity."                                │
│      },                                                                                                         │
│      {                                                                                                          │
│          "title": "Scaling Up Language Models: From Pre-training to Fine-tuning",                               │
│          "authors": ["Brown, T. B.", "Mann, B.", " Ryder, N.", "Subbiah, M.", "Kaplan, J.", "Dhariwal, P.",     │
│  "Neelakantan, A.", "Jones, A.", "Babbitt, J.", "Sutawika, P."],                                                │
│          "abstract": "In this paper, we explore the scaling of language models from pre-training to             │
│  fine-tuning phases. We present methods to significantly improve the performance of large language models by    │
│  examining alignment and task adaptation. Our benchmarking on various downstream tasks reveals that large       │
│  language models exhibit strong compositional generalization abilities and can effectively align to human       │
│  feedback, making them suitable for a wide range of applications in conversational agents and beyond."          │
│      },                                                                                                         │
│      {                                                                                                          │
│          "title": "AI Alignment: Why It Is Hard, and Where to Start",                                           │
│          "authors": ["Paul, D. A.", "Smith, J.", "Hughes, J.", "Sussman, G.", "Vaswani, A."],                   │
│          "abstract": "AI alignment remains one of the most pressing challenges in the development of powerful   │
│  AI systems, such as large language models. This paper offers an overview of the alignment problem, discussing  │
│  its inherent complexities and proposing initial methods for tackling it. We emphasize the role of              │
│  incorporating human values, long-term safety considera

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: CriticAgent                                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  | Paper Title                                      | SummarizerAgent Output                                    │
│  | Novelty Rating (1-5) | Clarity Rating (1-5) | Comments                                                       │
│  |                                                                                                              │
│  |--------------------------------------------------|---------------------------------------------------------  │
│  -------------------------------------------------------------------------------------------|-----------------  │
│  ----|---------------------|----------------------------------------------------------------------------------  │
│  ---------------------------------------------------------------------------|                                   │
│  | Language Models as Knowledge Bases?              | The paper investigates the potential of large             │
│  pre-trained language models (PLMs) to function as knowledge bases (KBs) and evaluates their effectiveness on   │
│  various knowledge-based tasks. The study compares the performance of PLMs against traditional KBs,             │
│  highlighting both strengths and limitations inherent in using language models for factual knowledge storage    │
│  and retrieval. | 4                   | 5                   | The summary captures the main focus and           │
│  implications of the study well. The comparison with traditional KBs adds clarity to its significance.          │
│  |                                                                                                              │
│  | Scaling Up Language Models: From Pre-training to Fine-tuning | The authors explore methods to enhance large  │
│  language models during the transition from pre-training to fine-tuning, focusing on performance improvements,  │
│  alignment, and task adaptation. They reveal that large language models show strong abilities in compositional  │
│  generalization and can align effectively with human feedback, showcasing their potential across various        │
│  applications in conversational agents. | 4                   | 5                   | The summary outlines the  │
│  objectives clearly, detailing both the transition phases and findings. It effectively highlights the           │
│  innovative aspects of the study. |                                                                             │
│  | AI Alignment: Why It Is Hard, and Where to Start | This paper discusses the complexity of the AI alignment   │
│  problem within powerful AI systems, emphasizing the necessity for incorporating human values and safety        │
│  considerations in alignment strategies. The authors propose methods to approach the alignment challenge,       │
│  establishing a foundational understanding for future research aimed at ensuring language models yield          │
│  beneficial outcomes. | 3                   | 4                   | The summary conveys the complexity of the   │
│  alignment issue well but could benefit from more specifics on the proposed methods to enhance clarity.         │
│  |                                                                                                              │
│  | Understanding GPT-3’s Responses: On the Behavior of Large Language Models | The study examines how changes   │
│  in model architecture and training techniques affect o

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f885da23-2e99-41e6-a76d-6a42e0185ca3                                                                     │
│  Agent: CriticAgent                                                                                             │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: f628b98a-ac4b-4048-8603-c2251398017e                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: | Paper Title                                      | SummarizerAgent Output                      │
│  | Novelty Rating (1-5) | Clarity Rating (1-5) | Comments                                                       │
│  |                                                                                                              │
│  |--------------------------------------------------|---------------------------------------------------------  │
│  -------------------------------------------------------------------------------------------|-----------------  │
│  ----|---------------------|----------------------------------------------------------------------------------  │
│  ---------------------------------------------------------------------------|                                   │
│  | Language Models as Knowledge Bases?              | The paper investigates the potential of large             │
│  pre-trained language models (PLMs) to function as knowledge bases (KBs) and evaluates their effectiveness on   │
│  various knowledge-based tasks. The study compares the performance of PLMs against traditional KBs,             │
│  highlighting both strengths and limitations inherent in using language models for factual knowledge storage    │
│  and retrieval. | 4                   | 5                   | The summary captures the main focus and           │
│  implications of the study well. The comparison with traditional KBs adds clarity to its significance.          │
│  |                                                                                                              │
│  | Scaling Up Language Models: From Pre-training to Fine-tuning | The authors explore methods to enhance large  │
│  language models during the transition from pre-training to fine-tuning, focusing on performance improvements,  │
│  alignment, and task adaptation. They reveal that large language models show strong abilities in compositional  │
│  generalization and can align effectively with human feedback, showcasing their potential across various        │
│  applications in conversational agents. | 4                   | 5                   | The summary outlines the  │
│  objectives clearly, detailing both the transition phases and findings. It effectively highlights the           │
│  innovative aspects of the study. |                                                                             │
│  | AI Alignment: Why It Is Hard, and Where to Start | This paper discusses the complexity of the AI alignment   │
│  problem within powerful AI systems, emphasizing the necessity for incorporating human values and safety        │
│  considerations in alignment strategies. The authors propose methods to approach the alignment challenge,       │
│  establishing a foundational understanding for future research aimed at ensuring language models yield          │
│  beneficial outcomes. | 3                   | 4                   | The summary conveys the complexity of the   │
│  alignment issue well but could benefit from more specifics on the proposed methods to enhance clarity.         │
│  |                                                                                                              │
│  | Understanding GPT-3’s Responses: On the Behavior of

######################
FINAL REPORT:
| Paper Title                                      | SummarizerAgent Output                                                                                                                            | Novelty Rating (1-5) | Clarity Rating (1-5) | Comments                                                                                                                                                      |
|--------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------|---------------------|---------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Language Models as Knowledge Bases?              | The paper investigates the potential of large pre-trained language models (PLMs) to function as kno

In [1]:
# 02_multi_agent.py
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool

load_dotenv()

search_tool = SerperDevTool()

# ============================================================
# AGENTS
# ============================================================

researcher = Agent(
    role="Senior Research Analyst",
    goal="Find the most relevant and influential academic papers on a topic",
    backstory="""You are a meticulous researcher who knows how to find 
    high-quality academic sources. You prioritize peer-reviewed papers 
    from top venues and pay attention to citation counts as signals of 
    influence. You cast a wide net initially, then filter ruthlessly 
    for relevance.""",
    tools=[search_tool],
    verbose=True
)

summarizer = Agent(
    role="Technical Writer",
    goal="Transform complex research findings into clear, accessible summaries",
    backstory="""You are a skilled technical writer who can distill complex 
    academic papers into clear insights. You focus on the 'so what'—not just 
    what the paper did, but why it matters. You write for an educated but 
    non-specialist audience, avoiding jargon where possible and explaining 
    it where necessary.""",
    verbose=True
)

critic = Agent(
    role="Quality Reviewer",
    goal="Evaluate summaries for accuracy, clarity, and completeness",
    backstory="""You are a demanding editor who holds content to high standards. 
    You check whether summaries accurately represent the source material, whether 
    they're clear to the target audience, and whether they capture the most 
    important points. You provide specific, actionable feedback.""",
    verbose=True
)

# ============================================================
# TASKS
# ============================================================

research_task = Task(
    description="""Search for the top 3 most relevant academic papers on: {topic}
    
    For each paper, provide:
    - Title (exact)
    - Authors (first author et al. for many authors)
    - Year and venue
    - Abstract or key findings (verbatim if possible, or close paraphrase)
    
    Focus on influential, well-cited papers from reputable venues.
    Include a mix of foundational and recent work if appropriate.
    """,
    expected_output="""A structured list of exactly 3 papers with full metadata 
    and abstracts/findings that the summarizer can work from.""",
    agent=researcher
)

summarize_task = Task(
    description="""Based on the research provided, create a summary for each paper.
    
    For each paper, write:
    - A 2-3 sentence summary of the main contribution
    - The key methodology or approach used
    - The primary results or findings
    - One sentence on broader implications
    
    Write for a technical audience who may not be specialists in this exact area.
    Be precise but accessible. Avoid unexplained jargon.
    """,
    expected_output="""Three paper summaries, each containing: main contribution,
    methodology, results, and implications. Clear, accurate, and accessible.""",
    agent=summarizer
)

critique_task = Task(
    description="""Review the summaries provided and evaluate each one.
    
    For each summary, rate (1-5 scale):
    - **Accuracy**: Does the summary correctly represent the paper's claims?
    - **Clarity**: Is it understandable to the target audience?
    - **Completeness**: Are the key points covered?
    
    Provide specific feedback on what could be improved.
    Be constructive but honest—this feedback will guide improvements.
    """,
    expected_output="""For each of the 3 summaries: numerical ratings for accuracy, 
    clarity, and completeness (1-5 each), plus specific written feedback.""",
    agent=critic
)

# ============================================================
# CREW
# ============================================================

crew = Crew(
    agents=[researcher, summarizer, critic],
    tasks=[research_task, summarize_task, critique_task],
    process=Process.sequential,
    verbose=True
)

if __name__ == "__main__":
    topic = "large language model alignment"
    print(f"\n🔍 Starting multi-agent research pipeline for: {topic}\n")
    
    result = crew.kickoff(inputs={"topic": topic})
    
    print("\n" + "="*60)
    print("FINAL OUTPUT (with critique)")
    print("="*60)
    print(result.raw)


🔍 Starting multi-agent research pipeline for: large language model alignment



╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e756ffed-75c1-4eac-ae79-ca4bf689a335                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Task: Search for the top 3 most relevant academic papers on: large language model alignment                    │
│                                                                                                                 │
│      For each paper, provide:                                                                                   │
│      - Title (exact)                                                                                            │
│      - Authors (first author et al. for many authors)                                                           │
│      - Year and venue                                                                                           │
│      - Abstract or key findings (verbatim if possible, or close paraphrase)                                     │
│                                                                                                                 │
│      Focus on influential, well-cited papers from reputable venues.                                             │
│      Include a mix of foundational and recent work if appropriate.                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Research Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Action: Search the internet with Serper                                                               │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Quality Reviewer                                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Summary Review                                                                                             │
│                                                                                                                 │
│  #### 1. **Diverse AI Feedback For Large Language Model Alignment**                                             │
│     - **Accuracy**: 5                                                                                           │
│     - **Clarity**: 4                                                                                            │
│     - **Completeness**: 5                                                                                       │
│     **Feedback**: This summary accurately represents the main contributions of the paper, detailing the         │
│  framework and results effectively. The clarity is mostly on point; however, the phrase "enriched feedback      │
│  from diverse sources" could be simplified or explained further for a broader audience. Make sure to            │
│  explicitly state what kinds of stakeholders are involved in the feedback process to enhance understanding.     │
│  Overall, key points are well-covered.                                                                          │
│                                                                                                                 │
│  #### 2. **The Moral Alignment of Large Language Models**                                                       │
│     - **Accuracy**: 4                                                                                           │
│     - **Clarity**: 5                                                                                            │
│     - **Completeness**: 4                                                                                       │
│     **Feedback**: The summary correctly addresses the significance of investigating moral alignment in LLMs.    │
│  However, while it mentions effective strategies, it doesn't elaborate on what those strategies are, which      │
│  diminishes completeness. It would benefit from including specific examples or methodologies used to improve    │
│  moral frameworks, which would enhance clarity and completeness. Overall, it communicates the dissertation's    │
│  focus very well.                                                                                               │
│                                                                                                                 │
│  #### 3. **Scalable Alignment of Large Language Models Towards Truth Seeking, Complex Reasoning, and Human      │
│  Values**                                                                                                       │
│     - **Accuracy**: 5                                                                                           │
│     - **Clarity**: 5                                                                                            │
│     - **Completeness**: 5                                                                                       │
│     **Feedback**: This summary presents a thorough overview of the thesis, accurately conveying its main        │
│  contributions, methodologies, and implications. It clearly highlights the necessity for alignment with human   │
│  values and provides extensive details on the approach 

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 8daadee4-b30e-47c4-b1da-520b57461a20                                                                     │
│  Agent: Quality Reviewer                                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e756ffed-75c1-4eac-ae79-ca4bf689a335                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### Summary Review                                                                               │
│                                                                                                                 │
│  #### 1. **Diverse AI Feedback For Large Language Model Alignment**                                             │
│     - **Accuracy**: 5                                                                                           │
│     - **Clarity**: 4                                                                                            │
│     - **Completeness**: 5                                                                                       │
│     **Feedback**: This summary accurately represents the main contributions of the paper, detailing the         │
│  framework and results effectively. The clarity is mostly on point; however, the phrase "enriched feedback      │
│  from diverse sources" could be simplified or explained further for a broader audience. Make sure to            │
│  explicitly state what kinds of stakeholders are involved in the feedback process to enhance understanding.     │
│  Overall, key points are well-covered.                                                                          │
│                                                                                                                 │
│  #### 2. **The Moral Alignment of Large Language Models**                                                       │
│     - **Accuracy**: 4                                                                                           │
│     - **Clarity**: 5                                                                                            │
│     - **Completeness**: 4                                                                                       │
│     **Feedback**: The summary correctly addresses the significance of investigating moral alignment in LLMs.    │
│  However, while it mentions effective strategies, it doesn't elaborate on what those strategies are, which      │
│  diminishes completeness. It would benefit from including specific examples or methodologies used to improve    │
│  moral frameworks, which would enhance clarity and completeness. Overall, it communicates the dissertation's    │
│  focus very well.                                                                                               │
│                                                                                                                 │
│  #### 3. **Scalable Alignment of Large Language Models Towards Truth Seeking, Complex Reasoning, and Human      │
│  Values**                                                                                                       │
│     - **Accuracy**: 5                                                                                           │
│     - **Clarity**: 5                                                                                            │
│     - **Completeness**: 5                                                                                       │
│     **Feedback**: This summary presents a thorough overview of the thesis, accurately conveying its main        │
│  contributions, methodologies, and implications. It cl


FINAL OUTPUT (with critique)
### Summary Review

#### 1. **Diverse AI Feedback For Large Language Model Alignment**  
   - **Accuracy**: 5  
   - **Clarity**: 4  
   - **Completeness**: 5  
   **Feedback**: This summary accurately represents the main contributions of the paper, detailing the framework and results effectively. The clarity is mostly on point; however, the phrase "enriched feedback from diverse sources" could be simplified or explained further for a broader audience. Make sure to explicitly state what kinds of stakeholders are involved in the feedback process to enhance understanding. Overall, key points are well-covered.

#### 2. **The Moral Alignment of Large Language Models**  
   - **Accuracy**: 4  
   - **Clarity**: 5  
   - **Completeness**: 4  
   **Feedback**: The summary correctly addresses the significance of investigating moral alignment in LLMs. However, while it mentions effective strategies, it doesn't elaborate on what those strategies are, which diminishe